## 파일 불러오기, DB 연동

#### ▶ 파일 입출력

- 쓰기

In [5]:
# w : 없으면 생성, 있으면 삭제 후 재생성(새로쓰기)
# a : 기존 파일을 열어 추가(추가쓰기)

f = open('C:/STUDY/StudyPython_sung/temp.txt', mode='w', encoding='utf-8')

f.write('안녕하세요.\n')
f.write('저는 허아현입니다.\n')
f.write('한국사람이죠.\n')

# 필수 (파일, 데이터베이스, 네트워크는 연결 후 반드시 클로징 필요)
f.close()     

print('파일 생성 완료')

파일 생성 완료


- 읽기

In [4]:
f=open('C:/STUDY/StudyPython_sung/temp.txt', mode='r', encoding='utf-8')

# t = f.read()     # 보통 한번에 잘 안불러옴

while True :                # 무한루프
    line = f.readline()     # 한 줄씩 읽기
    if not line : break

    print(line, end = '')

# print(t)

f.close()    # 필수
print('\n파일 읽기 완료')

안녕하세요.
저는 허아현입니다.
한국사람이죠.

파일 읽기 완료


- CSV 파일 읽기

In [ ]:
import csv

file_name = 'C:/STUDY/StudyPython_sung/busanbus_220504.csv'

# 보통 노트패드에서 utf-8로 변경 후 작업
f = open(file_name, mode='r', encoding='utf-8')

reader = csv.reader(f, delimiter = ',')     # 구분자가 ,인 경우
next(reader)   # 첫 행이 제목줄(헤더)인 경우 읽지않음

for line in reader :
    print(line)

f.close()     ## 필수

#### ▶ DB 연동

- Oracle

In [ ]:
# 오라클DB 연동
import cx_Oracle as ora

# 데이터소스 네임 객체 생성(접속주소, 포트번호, 서비스명)
dsn = ora.makedsn('localhost', 1521, service_name = 'orcl')
# DB연결객체
conn = ora.connect(user = 'scott', password = 'tiger', dsn = dsn, encoding = 'UTF-8')

# 커서 생성
cur = conn.cursor()
# 쿼리문 생성 및 작성
query = 'SELECT * FROM membertbl'

for row in cur.execute(query) :
    print(row)

# DB 클로징 필수 (네트워크, DB, 파일 연결 후 클로징 필수)
cur.close()
conn.close()

##### ▷ 대표 예제

In [ ]:
# 오라클DB 연동
import cx_Oracle as ora

# DB 접속함수
def myconn() :
    # 데이터소스 네임 객체 생성(접속주소, 포트번호, 서비스명)
    dsn = ora.makedsn('localhost', 1521, service_name = 'orcl')
    # DB연결객체
    conn = ora.connect(user = 'scott', password = 'tiger', dsn = dsn, encoding = 'UTF-8')
    return conn

# 기본 조회(for)
def test01(conn) :
    cur = conn.cursor()
    query = 'SELECT * FROM emp'

    for row in cur.execute(query) :
        print(row)

    cur.close()
    conn.close()

# 조회(while, fetchone)
def test02(conn) :
    cur = conn.cursor()
    query = 'SELECT * FROM emp'
    cur.execute(query)

    while True :
        row = cur.fetchone()
        if row is None : break
        print(row)

    cur.close()
    conn.close()


if __name__ == '__main__' :
    print('--- SQL 조회 기본 실행 ---')
    test01(myconn())
    print('--- SQL 조회 fetchone 사용 ---')
    test02(myconn())


In [ ]:
# DB연동 - INSERT
import cx_Oracle as ora

def myconn() :
    dsn = ora.makedsn('localhost', 1521, service_name = 'orcl')
    conn = ora.connect(user = 'scott', password = 'tiger', dsn = dsn, encoding = 'utf-8')
    return conn

def get_list(conn) :
    cur = conn.cursor()
    for row in cur.execute('SELECT * FROM divtbl') :
        print(row)

    cur.close()
    conn.close()

def set_list(conn, tup) :
    cur = conn.cursor()
    # query = f"INSERT INTO divtbl VALUES ('{tup[0]}', '{tup[1]}')"  # 보안취약
    query = f"INSERT INTO divtbl VALUES (:1, :2)"  # SQL Inject 보안문제+사용효율성
    cur.execute(query)
    conn.commit()    # 완전 저장
    cur.close()
    conn.close()



if __name__ == '__main__' :
    print('DIVTBL 데이터 조회')
    get_list(myconn())
    print('DIVTBL 신규 입력')
    division, names = input('(divsion, names) 입력 (구분자 \' \') > ').split(' ')      # \' : 작은따옴표를 그대로 표현
    tup = (division, names)
    set_list(myconn(), tup)

print('입력후 DIVTBL 데이터 조회 ')
get_list(myconn())
